In [139]:
from datetime import datetime, timedelta
import pandas as pd
import yfinance

data_folder = '../data/lorentzian'
ticker = 'SOL-GBP'
interval = "15m"
#start_time = datetime.today() - timedelta(days=days)
#end_time = datetime.now()

window_size = 30
windows_ago = 0

start_time = datetime.now() - timedelta(days = window_size + (window_size * windows_ago))
end_time = datetime.now() - timedelta(days = window_size * windows_ago)

ticker_data = yfinance.Ticker(ticker)

df = ticker_data.history(interval=interval, start=start_time, end=end_time)

new_names = {'Open': 'open', 'High': 'high', 'Low': 'low', 'Close': 'close', 'Adj Close': 'adj_close', 'Volume': 'volume'}
df = df.rename(columns=new_names)

#df['date'] = pd.DatetimeIndex(df['Datetime'])
#df.set_index("Date", inplace=True)

#df = df[-2000:]
df

,open,high,low,close,volume,Dividends,Stock Splits
Datetime,,,,,,,
2024-05-04 00:45:00+00:00,115.368324,115.368324,115.172935,115.172935,0,0.0,0.0
2024-05-04 01:00:00+00:00,115.128883,115.128883,114.540588,114.671944,2502400,0.0,0.0
2024-05-04 01:15:00+00:00,114.656021,114.913605,114.519691,114.913605,768000,0.0,0.0
2024-05-04 01:30:00+00:00,114.910011,115.329811,114.693359,114.693359,2192640,0.0,0.0
2024-05-04 01:45:00+00:00,114.683022,114.683022,114.469475,114.495949,0,0.0,0.0
...,...,...,...,...,...,...,...
2024-06-02 23:45:00+00:00,128.275497,128.275497,128.039749,128.065613,281600,0.0,0.0
2024-06-03 00:00:00+00:00,128.078659,128.078659,127.919334,128.075226,632448,0.0,0.0
2024-06-03 00:15:00+00:00,128.089325,128.143661,127.433235,127.436882,388352,0.0,0.0


In [140]:
from advanced_ta import LorentzianClassification
from ta.volume import money_flow_index as MFI

# df here is the dataframe containing stock data as [['open', 'high', 'low', 'close', 'volume']]
lc = LorentzianClassification(
    df,    
    features=[
        LorentzianClassification.Feature("RSI", 14, 2),  # f1
        LorentzianClassification.Feature("WT", 10, 11),  # f2
        LorentzianClassification.Feature("CCI", 20, 2),  # f3
        LorentzianClassification.Feature("ADX", 20, 2),  # f4
        LorentzianClassification.Feature("RSI", 9, 2),   # f5
        MFI(df['high'], df['low'], df['close'], df['volume'], 14) #f6
    ],
    settings=LorentzianClassification.Settings(
        source=df['close'],
        neighborsCount=16,
        maxBarsBack=2000,
        useDynamicExits=False
    ),
    filterSettings=LorentzianClassification.FilterSettings(
        useVolatilityFilter=True,
        useRegimeFilter=True,
        useAdxFilter=True,
        regimeThreshold=0.1,
        adxThreshold=20,
        kernelFilter = LorentzianClassification.KernelFilter(
            useKernelSmoothing = False,
            lookbackWindow = 8,
            relativeWeight = 8.0,
            regressionLevel = 25,
            crossoverLag = 2
        )
    )
)

lc.dump(f'{data_folder}/{ticker}_result.csv')

In [151]:
# Performance calculations

trade_signals_df = lc.data[lc.data.apply(lambda x: not pd.isna(x['startLongTrade']) or not pd.isna(x['endLongTrade']) or not pd.isna(x['startShortTrade']) or not pd.isna(x['endShortTrade']), axis=1)]

first_price = lc.data.iloc[0]['open']
last_price = lc.data.tail(1).iloc[0]['open']

transcaction_cost_adjustment = .996
pot_percentage_to_bet = .5

initial_value = 200000
initial_cash = (initial_value / 2)
initial_shares = ((initial_value / 2) * transcaction_cost_adjustment) / first_price

print(f'Initial cash is £{initial_cash / 100} and initial shares are {initial_shares}')
print('\n---------------------\n')

cash = initial_cash
shares = initial_shares

longTradeSize = 0
shortTradeSize = 0

for index, row in trade_signals_df.iterrows():
  price = row['open']

  if pd.isna(row['endLongTrade']) == False and longTradeSize > 0:
    shares -= longTradeSize
    cash_value = longTradeSize * price * transcaction_cost_adjustment
    cash += cash_value

    print(f'Closed long trade for {longTradeSize} shares at {price}. Cashed £{cash_value / 100}, current cash is £{cash / 100} and current shares are {shares}')

    longTradeSize = 0
  elif pd.isna(row['endShortTrade']) == False and shortTradeSize > 0:    
    shares += shortTradeSize
    cash_value = shortTradeSize * price * transcaction_cost_adjustment
    cash -= cash_value

    print(f'Closed short trade for {shortTradeSize} shares at {price}. Paid £{cash_value / 100}, current cash is £{cash / 100} and current shares are {shares}')

    shortTradeSize = 0
  elif pd.isna(row['startLongTrade']) == False and longTradeSize == 0:
    cash_value = cash * pot_percentage_to_bet
    cash -= cash_value
    longTradeSize = (cash_value * transcaction_cost_adjustment) / price
    shares += longTradeSize

    print(f'Opened long trade for {longTradeSize} shares at {price}. Paid £{cash_value / 100}, current cash is £{cash / 100} and current shares are {shares}')
  elif pd.isna(row['startShortTrade']) == False and shortTradeSize == 0:
    shortTradeSize = shares * pot_percentage_to_bet
    shares -= shortTradeSize
    cash_value = shortTradeSize * price * transcaction_cost_adjustment
    cash += cash_value

    print(f'Opened short trade for {shortTradeSize} shares at {price}. Cashed £{cash_value / 100}, current cash is £{cash / 100} and current shares are {shares}')
  else:
    continue

value = cash +  (shares * last_price) * transcaction_cost_adjustment

print('\n---------------------\n')
print(f'Initial value: {initial_value / 100}')
print(f'Final value: {value / 100}')
print(f'Portfolio growth: {((value - initial_value) / initial_value) * 100} %')
print(f'Asset growth: {((last_price - first_price) / first_price) * 100} %')

Initial cash is £1000.0 and initial shares are 863.3218920511955

---------------------

Opened long trade for 442.696809054653 shares at 112.49234008789062. Paid £500.0, current cash is £500.0 and current shares are 1306.0187011058485
Closed long trade for 442.696809054653 shares at 114.76609802246094. Cashed £506.0335904066819, current cash is £1006.0335904066819 and current shares are 863.3218920511956
Opened short trade for 431.6609460255978 shares at 117.2723159790039. Cashed £504.1939134267722, current cash is £1510.2275038334542 and current shares are 431.6609460255978
Closed short trade for 431.6609460255978 shares at 117.39120483398438. Paid £504.7050573958753, current cash is £1005.5224464375788 and current shares are 863.3218920511956
Opened long trade for 425.74597201674584 shares at 117.61712646484375. Paid £502.7612232187894, current cash is £502.7612232187894 and current shares are 1289.0678640679414
Closed long trade for 425.74597201674584 shares at 120.50401306152344. 

Datetime
2024-05-04 00:45:00+00:00     0
2024-05-04 01:00:00+00:00     0
2024-05-04 01:15:00+00:00     0
2024-05-04 01:30:00+00:00     0
2024-05-04 01:45:00+00:00     0
                             ..
2024-06-02 23:45:00+00:00    -1
2024-06-03 00:00:00+00:00    -1
2024-06-03 00:15:00+00:00    -1
2024-06-03 00:30:00+00:00    -1
2024-06-03 00:45:00+00:00    -1
Name: signal, Length: 2879, dtype: object

In [ ]:
lc.plot(f'{data_folder}/{ticker}_result.jpg')